In [ ]:
import time

from Autoencoder import Autoencoder
from NeuralNetwork import NeuralNetwork

from Dense import Dense

from utils import *

In [ ]:
epoch_exp = [100, 500]
train_exp = [750, 1500, 3000, 7500, 10000, 15000]
# batch_exp = [2**x for x in range(9, 4, -1)]
batch_exp = [512, 256, 128, 64, 32]

repetitions = 3

start_time = time.time()

for epoch in epoch_exp:
    etime_batch = time.time()
    for train_size in train_exp:
        X_train, _, _, _ = load_MNIST(train_size = train_size, test_size = int(0.2*train_size))
        ttime_batch = time.time()
        for batch in batch_exp:
            stime_batch = time.time()
            for rep in range(repetitions):
                encoder = Autoencoder(lr = 1e-1)
                
                encoder.add(Dense(28*28, 350))
                encoder.add(Dense(350, 128, output_layer = True))
                
                encoder.fit(X_train, X_train, epochs = epoch, batch_size = batch, verbose = False)
                # report_encode(X_test, X_test, encoder)
                # encoder.plot_loss()
                encoder.save(f"experiments/encoder_epoch_{epoch}_{train_size}_{batch}_{rep}.pkl")
            print(f"Terminou batch {batch}:\t{time.time() - stime_batch:.2f}s")
        print('-----***-----'*10)
        print(f"Terminou treino {train_size}:\t{time.time() - ttime_batch:.2f}s")
        print('-----***-----'*10 + '\n')
    print('-----***-----'*10)
    print(f"\nTerminou epoca {epoch}:\t{time.time() - etime_batch:.2f}s")
    print(['//***//']*10 + ['\n'])
print('-----***-----'*10)
print(f"\nTempo total:\t{time.time() - start_time:.2f}s")
print('-----***-----'*10 + '\n')

In [ ]:
epoch_exp = [100, 500]
train_exp = [750, 1500, 3000, 7500, 10000]
# batch_exp = [2**x for x in range(9, 4, -1)]
batch_exp = [512, 256, 128, 64, 32]
repetitions = 3
losses = {}

columns = ['Batch Size', 'Epochs', 'Train Size', 'Train Time', 'MSE Train', 'rep', 'error_trace']
df = pd.DataFrame(columns = columns)


# Load models and retrieve losses
for epoch in epoch_exp:
    for train_size in train_exp:
        for batch in batch_exp:
            epoch_losses = []
            error_trace = []
            for rep in range(repetitions):
                filename = f"experiments/encoder_epoch_{epoch}_{train_size}_{batch}_{rep}.pkl"
                if os.path.exists(filename):
                    encoder = Autoencoder.load(filename)  # Load model
                    epoch_losses.append(encoder.mse_train)  # Retrieve last MSE from training
                    error_trace.append(encoder.err_trace)
                    data = {'Batch Size': encoder.batch,
                            'Epochs': encoder.epochs,
                            'Train Size': encoder.size_train,
                            'Train Time': encoder.time_train,
                            'MSE Train': encoder.mse_train,
                            'rep': rep,
                            'error_trace': np.array(encoder.err_trace)}
                    df.loc[len(df)] = data
            losses[(epoch, train_size, batch)] = ((np.mean(epoch_losses), np.std(epoch_losses, ddof=1)),)

df.loc[(df['Batch Size'] == 32) & 
       (df['Epochs'] == 100) & 
       (df['Train Size'] == 7500) & 
       (df['rep'] == 0), 'Train Time'] = 207.167867